In [ ]:
import subprocess
import sys

COLAB = "google.colab" in sys.modules


def _install(package):
    if COLAB:
        ans = input(f"Install { package }? [y/n]:")
        if ans.lower() in ["y", "yes"]:
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", "--quiet", package]
            )
            print(f"{ package } installed!")


def _colab_install_missing_deps(deps):
    import importlib

    for dep in deps:
        if importlib.util.find_spec(dep) is None:
            if dep == "iris":
                dep = "scitools-iris"
            _install(dep)


deps = ["compliance-checker"]
_colab_install_missing_deps(deps)

# Shore Station Compliance Checker Script

Created: 2017-05-14

The IOOS Compliance Checker is a Python-based tool that helps users check the meta data compliance of a netCDF file. This software can be run in a web interface here: [https://compliance.ioos.us/index.html](https://compliance.ioos.us/index.html) The checker can also be run as a Python tool either on the command line or in a Python script.  This notebook demonstrates the python usage of the Compliance Checker.

## Purpose:

Run the compliance checker python tool on a Scipps Pier shore station dataset to check for the metadata compliance.

The Scripps Pier automated shore station operated by Southern California Coastal Ocean Observing System (SCCOOS) at Scripps Institution of Oceanography (SIO) is mounted at a nominal depth of 5 meters MLLW. The instrument package includes a Seabird SBE 16plus SEACAT Conductivity, Temperature, and Pressure recorder, and a Seapoint Chlorophyll Fluorometer with a 0-50 ug/L gain setting.

## Dependencies:

This script must be run in the "IOOS" environment for the compliance checker to work properly.

Written by: J.Bosch Feb. 10, 2017

In [1]:
import compliance_checker

print(compliance_checker.__version__)

4.3.4


In [2]:
# First import the compliance checker and test that it is installed properly.
from compliance_checker.runner import CheckSuite, ComplianceChecker

# Load all available checker classes.
check_suite = CheckSuite()
check_suite.load_all_available_checkers()

In [3]:
# Path to the Scripps Pier Data.

url = "http://gliders.ioos.us/thredds/dodsC/deployments/rutgers/ru29-20150623T1046/ru29-20150623T1046.nc3.nc"

### Running Compliance Checker on the Scripps Pier shore station data

This code is written with all the arguments spelled out, following the usage instructions on the README section of compliance checker github page: https://github.com/ioos/compliance-checker

In [4]:
output_file = "buoy_testCC.txt"

return_value, errors = ComplianceChecker.run_checker(
    ds_loc=url,
    checker_names=["cf", "acdd"],
    verbose=True,
    criteria="normal",
    skip_checks=None,
    output_filename=output_file,
    output_format="text",
)

In [5]:
with open(output_file) as f:
    print(f.read())



--------------------------------------------------------------------------------
                         IOOS Compliance Checker Report                         
                                 Version 4.3.4                                  
                     Report generated 2022-01-14T13:19:59Z                      
                                     cf:1.7                                     
http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html
--------------------------------------------------------------------------------
                               Corrective Actions                               
ru29-20150623T1046.nc3.nc#fillmismatch has 5 potential issues


                                     Errors                                     
--------------------------------------------------------------------------------
§3.5 Flags
* precise_lat_qc's flag_meanings and flag_values should have the same number of elements.
* precise_lon_qc's f

This Compliance Checker Report can be used to identify where file meta data can be improved.  A strong meta data record allows for greater utility of the data for a broader audience of data analysts.

In [6]:
import requests
test = "acdd"

cc_url = f"https://data.ioos.us/compliance/api/run?report_format=html&test={test}&url={url}"
print(cc_url)


r = requests.get(cc_url)
print(r.content)

https://data.ioos.us/compliance/api/run?report_format=html&test=acdd&url=http://gliders.ioos.us/thredds/dodsC/deployments/rutgers/ru29-20150623T1046/ru29-20150623T1046.nc3.nc#fillmismatch
b'{"error":"OSError","message":"[Errno -45] NetCDF: Not a valid data type or _FillValue type mismatch: b\'https://gliders.ioos.us/thredds/dodsC/deployments/rutgers/ru29-20150623T1046/ru29-20150623T1046.nc3.nc\'"}\n'


In [7]:
test = "cf"

cc_url = f"https://data.ioos.us/compliance/api/run?report_format=html&test={test}&url={url}"
print(cc_url)


r = requests.get(cc_url)
print(r.content)

https://data.ioos.us/compliance/api/run?report_format=html&test=cf&url=http://gliders.ioos.us/thredds/dodsC/deployments/rutgers/ru29-20150623T1046/ru29-20150623T1046.nc3.nc#fillmismatch
b'{"error":"OSError","message":"[Errno -45] NetCDF: Not a valid data type or _FillValue type mismatch: b\'https://gliders.ioos.us/thredds/dodsC/deployments/rutgers/ru29-20150623T1046/ru29-20150623T1046.nc3.nc\'"}\n'
